<a href="https://colab.research.google.com/github/wusinyee/MW-Portfolio/blob/main/TCCADBv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install jupyter-dash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.2 MB/s eta 0:00:00


In [ ]:
!pip install streamlit plotly pandas numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.3 MB/s eta 0:00:00


In [1]:
!pip install dash plotly pandas numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 23.8 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Load the data
try:
    df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')
except FileNotFoundError:
    print("Error: Data file not found. Please check the file path.")
    raise

# Data preprocessing
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['TotalCharges'].fillna(df['TotalCharges'].mean(), inplace=True)
df['Churn'] = df['Churn'].map({'Yes': 'Churned', 'No': 'Retained'})

# Define color palette
color_palette = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']

# Functions for plotting
def plot_churn_distribution(data):
    return px.pie(data, names='Churn', title='Overall Churn Distribution',
                  color='Churn', color_discrete_sequence=color_palette[:2],
                  hole=0.3, labels={'Churn': 'Customer Status'})

def plot_categorical_distribution(data, feature):
    return px.histogram(data, x=feature, color='Churn', barmode='group',
                        title=f'Distribution of {feature}',
                        color_discrete_sequence=color_palette[:2],
                        labels={feature: feature.replace('_', ' ').title()})

def plot_churn_rate_by_category(data, feature):
    churn_rate = data.groupby(feature)['Churn'].apply(lambda x: (x == 'Churned').mean()).reset_index()
    churn_rate.columns = [feature, 'Churn_Rate']
    return px.bar(churn_rate, x=feature, y='Churn_Rate',
                  title=f'Churn Rate by {feature}',
                  color='Churn_Rate', color_continuous_scale=color_palette[:3],
                  labels={feature: feature.replace('_', ' ').title(), 'Churn_Rate': 'Churn Rate'})

def plot_numerical_distribution(data, feature):
    return px.histogram(data, x=feature, color='Churn', marginal='box',
                        title=f'Distribution of {feature}',
                        color_discrete_sequence=color_palette[:2],
                        labels={feature: feature.replace('_', ' ').title()})

def plot_correlation_heatmap(data):
    numeric_df = data.select_dtypes(include=[np.number])
    corr = numeric_df.corr()
    return px.imshow(corr, title='Correlation Heatmap',
                     color_continuous_scale=color_palette[:3],
                     labels=dict(color="Correlation"))

def plot_tenure_vs_charges(data):
    return px.scatter(data, x='tenure', y='MonthlyCharges', color='Churn',
                      title='Tenure vs Monthly Charges',
                      labels={'tenure': 'Tenure (months)',
                              'MonthlyCharges': 'Monthly Charges ($)'},
                      color_discrete_sequence=color_palette[:2],
                      hover_data=['Contract', 'TotalCharges'],
                      size='TotalCharges', opacity=0.7)

# Create dashboard
def create_dashboard(categorical_feature, numerical_feature):
    fig = make_subplots(rows=2, cols=3,
                        subplot_titles=('Churn Distribution', 'Correlation Heatmap',
                                        f'Distribution of {categorical_feature}', f'Churn Rate by {categorical_feature}',
                                        f'Distribution of {numerical_feature}', 'Tenure vs Monthly Charges'),
                        specs=[[{'type': 'domain'}, {'type': 'heatmap'}, {'type': 'xy'}],
                               [{'type': 'xy'}, {'type': 'xy'}, {'type': 'xy'}]],
                        vertical_spacing=0.1,
                        horizontal_spacing=0.05)

    # Add plots to the dashboard
    fig.add_trace(plot_churn_distribution(df).data[0], row=1, col=1)
    fig.add_trace(plot_correlation_heatmap(df).data[0], row=1, col=2)
    fig.add_trace(plot_categorical_distribution(df, categorical_feature).data[0], row=1, col=3)
    fig.add_trace(plot_churn_rate_by_category(df, categorical_feature).data[0], row=2, col=1)
    fig.add_trace(plot_numerical_distribution(df, numerical_feature).data[0], row=2, col=2)
    fig.add_trace(plot_tenure_vs_charges(df).data[0], row=2, col=3)

    # Update layout
    fig.update_layout(
        height=1200,
        width=1800,
        title={
            'text': "Telco Customer Churn Analysis Dashboard",
            'y':0.98,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': dict(size=24, family='Arial', color='#333333')
        },
        plot_bgcolor='white',
        paper_bgcolor='white',
        font=dict(family="Arial", size=12, color="#333333"),
        margin=dict(l=60, r=60, t=120, b=60),
        hovermode="closest",
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=-0.15,
            xanchor="center",
            x=0.5
        )
    )

    # Adjust color bar for heatmap
    fig.update_layout(coloraxis_colorbar=dict(
        lenmode="fraction", len=0.75,
        yanchor="top", y=1,
        xanchor="left", x=1.02,
        title="Correlation"
    ))

    # Update axes and titles
    for i in fig['layout']['annotations']:
        i['font'] = dict(size=14, color='#333333')

    # Adjust individual subplot layouts
    fig.update_xaxes(title_standoff=25, showgrid=True, gridwidth=1, gridcolor='#E5E5E5', title_font=dict(size=12))
    fig.update_yaxes(title_standoff=25, showgrid=True, gridwidth=1, gridcolor='#E5E5E5', title_font=dict(size=12))

    # Ensure pie chart is circular
    fig.update_layout(scene_aspectmode='data')

    fig.show()

# Get list of features
categorical_features = [col for col in df.select_dtypes(include=['object']).columns if col not in ['customerID', 'Churn']]
numerical_features = df.select_dtypes(include=[np.number]).columns.tolist()

# Display initial dashboard
create_dashboard('InternetService', 'tenure')

# Function to update plots
def update_plots():
    print("\nAvailable categorical features:", ", ".join(categorical_features))
    print("Available numerical features:", ", ".join(numerical_features))

    while True:
        categorical_feature = input("\nEnter the categorical feature: ")
        if categorical_feature in categorical_features:
            break
        print("Invalid feature. Please choose from the available categorical features.")

    while True:
        numerical_feature = input("Enter the numerical feature: ")
        if numerical_feature in numerical_features:
            break
        print("Invalid feature. Please choose from the available numerical features.")

    create_dashboard(categorical_feature, numerical_feature)

print("\nTo update the dashboard, run the update_plots() function and enter new features when prompted.")


To update the dashboard, run the update_plots() function and enter new features when prompted.


In [4]:
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
import pandas as pd

# Load the data
df = pd.read_csv('/content/WA_Fn-UseC_-Telco-Customer-Churn.csv')

# Define the color palette
colors = {
    'background': '#F0F8FF',  # Alice Blue
    'accent': '#1E90FF',      # Dodger Blue
    'text': '#333333',        # Dark Gray
    'highlight': '#87CEFA'    # Light Sky Blue
}

# Create the dashboard layout
fig = make_subplots(
    rows=3, cols=3,
    subplot_titles=(
        'Churn Distribution', 'Tenure vs Monthly Charges', 'Distribution of Tenure',
        'Churn by Contract Type', 'Internet Service Distribution', 'Correlation Heatmap',
        'Churn Rate by Gender', 'Churn Rate by Senior Citizen', 'Customer Demographics'
    ),
    specs=[
        [{'type': 'domain'}, {'type': 'xy'}, {'type': 'xy'}],
        [{'type': 'xy'}, {'type': 'domain'}, {'type': 'xy'}],
        [{'type': 'xy'}, {'type': 'xy'}, {'type': 'table'}]
    ],
    vertical_spacing=0.1,
    horizontal_spacing=0.05,
)

# 1. Churn Distribution
churn_counts = df['Churn'].value_counts()
fig.add_trace(go.Pie(labels=churn_counts.index, values=churn_counts.values,
                     marker_colors=[colors['accent'], colors['highlight']],
                     name='Churn Distribution'),
              row=1, col=1)

# 2. Tenure vs Monthly Charges
fig.add_trace(go.Scatter(x=df['tenure'], y=df['MonthlyCharges'], mode='markers',
                         marker=dict(color=colors['accent'], size=5),
                         name='Tenure vs Monthly Charges'),
              row=1, col=2)

# 3. Distribution of Tenure
fig.add_trace(go.Histogram(x=df['tenure'], marker_color=colors['accent'],
                           name='Tenure Distribution'),
              row=1, col=3)

# 4. Churn by Contract Type
churn_by_contract = df.groupby('Contract')['Churn'].value_counts(normalize=True).unstack()
fig.add_trace(go.Bar(x=churn_by_contract.index, y=churn_by_contract['Yes'],
                     name='Churned', marker_color=colors['accent']),
              row=2, col=1)
fig.add_trace(go.Bar(x=churn_by_contract.index, y=churn_by_contract['No'],
                     name='Retained', marker_color=colors['highlight']),
              row=2, col=1)

# 5. Internet Service Distribution
internet_service = df['InternetService'].value_counts()
fig.add_trace(go.Pie(labels=internet_service.index, values=internet_service.values,
                     marker_colors=[colors['accent'], colors['highlight'], '#ADD8E6'],
                     name='Internet Service'),
              row=2, col=2)

# 6. Correlation Heatmap
numeric_df = df.select_dtypes(include=['float64', 'int64'])
correlation = numeric_df.corr()
fig.add_trace(go.Heatmap(z=correlation.values, x=correlation.columns, y=correlation.columns,
                         colorscale='Blues', name='Correlation Heatmap'),
              row=2, col=3)

# 7. Churn Rate by Gender
churn_by_gender = df.groupby('gender')['Churn'].value_counts(normalize=True).unstack()
fig.add_trace(go.Bar(x=churn_by_gender.index, y=churn_by_gender['Yes'],
                     name='Churn Rate by Gender', marker_color=colors['accent']),
              row=3, col=1)

# 8. Churn Rate by Senior Citizen
churn_by_senior = df.groupby('SeniorCitizen')['Churn'].value_counts(normalize=True).unstack()
fig.add_trace(go.Bar(x=['Non-Senior', 'Senior'], y=churn_by_senior['Yes'],
                     name='Churn Rate by Senior Citizen', marker_color=colors['accent']),
              row=3, col=2)

# 9. Customer Demographics table
demographics = df[['gender', 'SeniorCitizen', 'Partner', 'Dependents']].describe().T
fig.add_trace(
    go.Table(
        header=dict(values=['Metric'] + list(demographics.columns),
                    fill_color=colors['highlight'],
                    align='left'),
        cells=dict(values=[demographics.index] + [demographics[col] for col in demographics.columns],
                   fill_color=colors['background'],
                   align='left')
    ),
    row=3, col=3
)

# Update layout
fig.update_layout(
    height=1200,
    width=1600,
    showlegend=False,
    title=dict(
        text="Telco Customer Churn Dashboard",
        x=0.5,
        y=0.98,
        xanchor='center',
        yanchor='top',
        font=dict(size=24)
    ),
    font=dict(family="Arial, sans-serif", size=12, color=colors['text']),
    plot_bgcolor=colors['background'],
    paper_bgcolor=colors['background'],
    barmode='stack',
    updatemenus=[
        dict(
            buttons=list([
                dict(label="All",
                     method="update",
                     args=[{"visible": [True] * len(fig.data)},
                           {"title": "Telco Customer Churn Dashboard"}]),
                dict(label="Churn Distribution",
                     method="update",
                     args=[{"visible": [True] + [False] * (len(fig.data) - 1)},
                           {"title": "Churn Distribution"}]),
                # Add more buttons for other charts as needed
            ]),
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.1,
            xanchor="left",
            y=1.1,
            yanchor="top"
        ),
    ]
)

# Update axes
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor=colors['highlight'])
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor=colors['highlight'])

# Update subplot titles
for i in fig['layout']['annotations']:
    i['font'] = dict(size=14, color=colors['text'])

# Show the figure
fig.show()

In [6]:
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np

# Load the data
df = pd.read_csv('/content/WA_Fn-UseC_-Telco-Customer-Churn.csv')

# Define the updated theme
theme = {
    "accent": "#4a90e2",
    "accent_negative": "#e74c3c",
    "accent_positive": "#2ecc71",
    "background_content": "rgb(250,250,250)",
    "background_page": "rgb(240,240,240)",
    "border": "rgb(220,220,220)",
    "font_family": "Arial, sans-serif",
    "font_size": "14px",
    "font_family_header": "Arial, sans-serif",
    "font_size_header": "24px",
    "font_family_headings": "Arial, sans-serif",
    "text": "rgb(50,50,50)",
    "colorway": [
        "#4a90e2", "#e74c3c", "#2ecc71", "#f39c12", "#9b59b6",
        "#3498db", "#e67e22", "#1abc9c", "#34495e", "#95a5a6"
    ],
    "colorscale": [
        "#f7fbff", "#deebf7", "#c6dbef", "#9ecae1", "#6baed6",
        "#4292c6", "#2171b5", "#08519c", "#08306b", "#081d58"
    ]
}

# Create the dashboard layout
fig = make_subplots(
    rows=3, cols=3,
    subplot_titles=(
        'Churn Distribution', 'Tenure vs Monthly Charges', 'Distribution of Tenure',
        'Churn by Contract Type', 'Internet Service Distribution', 'Correlation Heatmap',
        'Churn Rate by Gender', 'Churn Rate by Senior Citizen', 'Monthly Charges Distribution'
    ),
    specs=[
        [{'type': 'domain'}, {'type': 'xy'}, {'type': 'xy'}],
        [{'type': 'xy'}, {'type': 'domain'}, {'type': 'xy'}],
        [{'type': 'xy'}, {'type': 'xy'}, {'type': 'xy'}]
    ],
    vertical_spacing=0.1,
    horizontal_spacing=0.05,
)

# 1. Churn Distribution
churn_counts = df['Churn'].value_counts()
churn_rate = churn_counts['Yes'] / len(df) * 100
fig.add_trace(go.Pie(
    labels=churn_counts.index,
    values=churn_counts.values,
    marker_colors=[theme['accent'], theme['accent_negative']],
    name='Churn Distribution',
    hovertemplate="<b>%{label}</b><br>Count: %{value}<br>Percentage: %{percent:.1f}%<extra></extra>"
), row=1, col=1)

# 2. Tenure vs Monthly Charges
fig.add_trace(go.Scatter(
    x=df['tenure'],
    y=df['MonthlyCharges'],
    mode='markers',
    marker=dict(
        color=df['Churn'].map({'Yes': theme['accent_negative'], 'No': theme['accent']}),
        size=5,
        opacity=0.7
    ),
    name='Tenure vs Monthly Charges',
    hovertemplate="<b>Tenure:</b> %{x} months<br><b>Monthly Charges:</b> $%{y:.2f}<br><b>Churn:</b> %{marker.color}<extra></extra>"
), row=1, col=2)

# 3. Distribution of Tenure
fig.add_trace(go.Histogram(
    x=df['tenure'],
    marker_color=theme['accent'],
    name='Tenure Distribution',
    hovertemplate="<b>Tenure:</b> %{x} months<br><b>Count:</b> %{y}<extra></extra>"
), row=1, col=3)

# 4. Churn by Contract Type
churn_by_contract = df.groupby('Contract')['Churn'].value_counts(normalize=True).unstack()
fig.add_trace(go.Bar(
    x=churn_by_contract.index,
    y=churn_by_contract['Yes'],
    name='Churned',
    marker_color=theme['accent_negative'],
    hovertemplate="<b>Contract:</b> %{x}<br><b>Churn Rate:</b> %{y:.1%}<extra></extra>"
), row=2, col=1)
fig.add_trace(go.Bar(
    x=churn_by_contract.index,
    y=churn_by_contract['No'],
    name='Retained',
    marker_color=theme['accent'],
    hovertemplate="<b>Contract:</b> %{x}<br><b>Retention Rate:</b> %{y:.1%}<extra></extra>"
), row=2, col=1)

# 5. Internet Service Distribution
internet_service = df['InternetService'].value_counts()
fig.add_trace(go.Pie(
    labels=internet_service.index,
    values=internet_service.values,
    marker_colors=[theme['colorway'][i] for i in range(len(internet_service))],
    name='Internet Service',
    hovertemplate="<b>%{label}</b><br>Count: %{value}<br>Percentage: %{percent:.1f}%<extra></extra>"
), row=2, col=2)

# 6. Correlation Heatmap
numeric_df = df.select_dtypes(include=['float64', 'int64'])
correlation = numeric_df.corr()
fig.add_trace(go.Heatmap(
    z=correlation.values,
    x=correlation.columns,
    y=correlation.columns,
    colorscale=theme['colorscale'],
    name='Correlation Heatmap',
    hovertemplate="<b>%{x}</b> vs <b>%{y}</b><br>Correlation: %{z:.2f}<extra></extra>"
), row=2, col=3)

# 7. Churn Rate by Gender
churn_by_gender = df.groupby('gender')['Churn'].value_counts(normalize=True).unstack()
fig.add_trace(go.Bar(
    x=churn_by_gender.index,
    y=churn_by_gender['Yes'],
    name='Churn Rate by Gender',
    marker_color=theme['accent_negative'],
    hovertemplate="<b>Gender:</b> %{x}<br><b>Churn Rate:</b> %{y:.1%}<extra></extra>"
), row=3, col=1)

# 8. Churn Rate by Senior Citizen
churn_by_senior = df.groupby('SeniorCitizen')['Churn'].value_counts(normalize=True).unstack()
fig.add_trace(go.Bar(
    x=['Non-Senior', 'Senior'],
    y=churn_by_senior['Yes'],
    name='Churn Rate by Senior Citizen',
    marker_color=theme['accent_negative'],
    hovertemplate="<b>Category:</b> %{x}<br><b>Churn Rate:</b> %{y:.1%}<extra></extra>"
), row=3, col=2)

# 9. Monthly Charges Distribution
fig.add_trace(go.Histogram(
    x=df['MonthlyCharges'],
    name='Monthly Charges Distribution',
    marker_color=theme['accent'],
    hovertemplate="<b>Monthly Charges:</b> $%{x:.2f}<br><b>Count:</b> %{y}<extra></extra>"
), row=3, col=3)

# Update layout
fig.update_layout(
    height=1080,
    width=1920,
    showlegend=False,
    title=dict(
        text="Telco Customer Churn Dashboard",
        x=0.5,
        y=0.98,
        xanchor='center',
        yanchor='top',
        font=dict(size=int(theme['font_size_header'].replace('px', '')), family=theme['font_family_header'])
    ),
    font=dict(family=theme['font_family'], size=int(theme['font_size'].replace('px', '')), color=theme['text']),
    plot_bgcolor=theme['background_content'],
    paper_bgcolor=theme['background_page'],
    barmode='stack',
)

# Update axes
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor=theme['border'])
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor=theme['border'])

# Update subplot titles
for i in fig['layout']['annotations']:
    i['font'] = dict(size=int(theme['font_size'].replace('px', ''))+2, family=theme['font_family_headings'], color=theme['text'])

# Add annotations for key insights
fig.add_annotation(
    x=0.5, y=1.05, xref="paper", yref="paper",
    text=f"Overall Churn Rate: {churn_rate:.1f}%",
    showarrow=False,
    font=dict(size=20, color=theme['accent_negative']),
    bgcolor="rgba(255, 255, 255, 0.7)",
    bordercolor=theme['border'],
    borderwidth=1,
    borderpad=4
)

fig.add_annotation(
    x=0.97, y=0.5, xref="paper", yref="paper",
    text="Key Factors:<br>1. Contract Type<br>2. Tenure<br>3. Monthly Charges",
    showarrow=False,
    font=dict(size=16),
    align="left",
    bgcolor="rgba(255, 255, 255, 0.7)",
    bordercolor=theme['border'],
    borderwidth=1,
    borderpad=4
)

# Show the figure
fig.show()